In [1]:
import json
from pymot.pymot import MOTEvaluation

# Load ground truth data from a JSON file
with open('gt_output.json', 'r') as f:
    groundtruth = json.load(f)

# Load hypotheses data from a JSON file
with open('yolo_output.json', 'r') as f:
    hypotheses = json.load(f)
    
overlap_threshold = 0.5
# Create an instance of the MOTEvaluation class with the loaded data
evaluator = MOTEvaluation(groundtruth, hypotheses,overlap_threshold)

# Calculate metrics
mota = evaluator.getMOTA()
motp = evaluator.getMOTP()
relative_statistics = evaluator.getRelativeStatistics()
absolute_statistics = evaluator.getAbsoluteStatistics()

KeyError: 'hypotheses'